In [1]:
%%time
%reload_ext autoreload
%autoreload
%autoreload 2
%config Completer.use_jedi = False

import os
import json
import warnings
import pprint
import sys
import numpy as np
import pandas as pd
import torch

MAIN_PATH = os.getcwd().split("notebooks")[0]
sys.path.insert(0, MAIN_PATH)

CPU times: user 2.03 s, sys: 478 ms, total: 2.51 s
Wall time: 1.85 s


In [3]:
# Run a Federated Learning experiment
#from data_loader.mnist import MnistDatasetManager
from data_loader.cifar10 import Cifar10DatasetManager
from models.mnist_model import MnistFullConnectModel, SimpleCNN
from server.base_server import BaseServer
from client.base_client import BaseClient
from experiments.base_experiment import BaseExperiment
from gradients.noise import GaussianNoiseGenerator, NoNoiseGenerator,StaircaseNoiseGenerator
from metrics.classification import multiclass_accuracy

In [5]:
cifar10_manager = Cifar10DatasetManager()


100.0%


Extracting /Users/shuyafeng/IdeaProjects/DifferentialPrivacy/datasets/cifar-10-python.tar.gz to /Users/shuyafeng/IdeaProjects/DifferentialPrivacy/datasets
Files already downloaded and verified


In [6]:
class DemoCifar10Experiment(BaseExperiment):
    def __init__(self, client_num: int =2):
        self._init_server_clients(client_num)
        self._init_data(client_num)

    
    def _init_server_clients(self, client_num):
        model = SimpleCNN
        self.clients = [BaseClient(model(), 
                                   client_id=idx, 
                                   noise_generator=StaircaseNoiseGenerator())
                        for idx in range(client_num)]
        self.server = BaseServer(model())

    def _init_data(self, client_num):
        data_manager = Cifar10DatasetManager(n_parties=client_num)
        self.client_train_datas = data_manager.train_loaders
        self.valid_datas = data_manager.validation_loader
        self.test_data = data_manager.test_loader

    def evaluate_model(self, data):
        total_correct = 0
        total_sample_num = 0
        with torch.no_grad():
            for _, (inputs, target) in enumerate(data):
                predict_labels = self.server.predict(inputs)
                correct, sample_num = multiclass_accuracy(y_pred=predict_labels, 
                                                          y_true=target)
                total_correct += correct
                total_sample_num += sample_num
                
        return total_correct / total_sample_num
        
    def get_validation_result(self):
        return self.evaluate_model(self.valid_datas)
    
    def get_test_result(self):
        return self.evaluate_model(self.test_data)
    
    def aggeragate(self):
        self.server.aggeragate_model(self.clients)
    
    def run(self, epochs: int):
        for client in self.clients:
            client.set_training_mode(for_gradient=False)

        for epoch in range(epochs):
            print(self.get_validation_result())
            for client, client_train_data in self.shuffled_data(to_shuffle=False):
                client.train(client_train_data)

            self.aggeragate()
            if epoch and not (epoch % 1):
                print(self.get_validation_result())

            self.distribute_model()


In [9]:
EXPERIMENT = DemoCifar10Experiment(client_num=10)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
EXPERIMENT.run(10)

RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[64, 3, 32, 32] to have 1 channels, but got 3 channels instead